## Import libraries 

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import predictor

## Load saved model

In [2]:
saved_model_dir= "./saved_model/1544257357"
predict_fn = predictor.from_saved_model(saved_model_dir)

INFO:tensorflow:Restoring parameters from ./saved_model/1544257357/variables/variables


In [3]:
def parse_func(s_example):
    features = {
                'workclass': tf.FixedLenFeature((), tf.string),
                'sex': tf.FixedLenFeature((), tf.string),
                'age': tf.FixedLenFeature((), tf.int64),
                'education': tf.FixedLenFeature((), tf.string),
                'education-num': tf.FixedLenFeature((), tf.int64),
                'marital-status': tf.FixedLenFeature((), tf.string),
                'occupation': tf.FixedLenFeature((), tf.string),
                'hours-per-week': tf.FixedLenFeature((), tf.int64),
                'label': tf.FixedLenFeature((), tf.string)
                 }
    return tf.parse_single_example(s_example, features=features)

In [4]:
data= tf.data.TFRecordDataset('census_test.tfrecord').batch(3)
next_batch= data.make_one_shot_iterator().get_next()
sess= tf.Session() 

## Read batches of serialized examples from `tfrecord` and feed them to predictor

In [5]:
examples = sess.run(next_batch)
predictions = predict_fn({'inputs':examples})

print(predictions['scores'],[sess.run(parse_func(example))['label'] for example in examples])

[[0.64114726 0.35885268]
 [0.44513756 0.55486244]
 [0.67866397 0.3213361 ]] [b' <=50K.', b' <=50K.', b' >50K.']


In [6]:
predictions

{'classes': array([[b'0', b'1'],
        [b'0', b'1'],
        [b'0', b'1']], dtype=object),
 'scores': array([[0.64114726, 0.35885268],
        [0.44513756, 0.55486244],
        [0.67866397, 0.3213361 ]], dtype=float32)}

In [7]:
sess.close()

### Inspect saved model meta data with `saved_model_cli show`

In [8]:
from tensorflow.python.tools import saved_model_cli

In [9]:
!saved_model_cli show --dir saved_model/1544257357 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['classification']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_example_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 2)
        name: head/Tile:0
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: head/predictions/probabilities:0
  Method name is: tensorflow/serving/classify

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['examples'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_example_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['class_ids'] tensor_info:
        dtype: DT_INT64
      

### Use `saved_model_cli run` to make inference against saved model (`predict` Signature)
#### First, save the batch of examples to `.npy` file
#### Then, use `saved_model_cli` to run the model by reading inputs from `.npy` file.

In [10]:
np.save('examples.npy',examples)

In [11]:
!saved_model_cli run --dir saved_model/1544257357 \
 --tag_set serve --signature_def predict \
 --inputs examples=examples.npy

2019-03-02 07:01:02.453741: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Result for output key class_ids:
[[0]
 [1]
 [0]]
Result for output key classes:
[[b'0']
 [b'1']
 [b'0']]
Result for output key logistic:
[[0.35885268]
 [0.55486244]
 [0.32133606]]
Result for output key logits:
[[-0.5803472 ]
 [ 0.22033691]
 [-0.7476385 ]]
Result for output key probabilities:
[[0.64114726 0.35885268]
 [0.44513756 0.55486244]
 [0.67866397 0.3213361 ]]


### Send inference request to the model serviced by tensorflow model server over REST api

In [11]:
import base64

encoded = np.array([base64.b64encode(example) for example in examples])
encoded

array([b'CoEDChQKBWxhYmVsEgsKCQoHIDw9NTBLLgoMCgNhZ2USBRoDCgEZChYKCWVkdWNhdGlvbhIJCgcKBSAxMXRoChcKDmhvdXJzLXBlci13ZWVrEgUaAwoBKAoQCgNzZXgSCQoHCgUgTWFsZQoeCgxyZWxhdGlvbnNoaXASDgoMCgogT3duLWNoaWxkChUKDGNhcGl0YWwtZ2FpbhIFGgMKAQAKJAoObmF0aXZlLWNvdW50cnkSEgoQCg4gVW5pdGVkLVN0YXRlcwokCgpvY2N1cGF0aW9uEhYKFAoSIE1hY2hpbmUtb3AtaW5zcGN0ChYKDWVkdWNhdGlvbi1udW0SBRoDCgEHCiQKDm1hcml0YWwtc3RhdHVzEhIKEAoOIE5ldmVyLW1hcnJpZWQKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIEJsYWNrChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA/LrDQ==',
       b'CoUDChwKDHJlbGF0aW9uc2hpcBIMCgoKCCBIdXNiYW5kChUKDGNhcGl0YWwtZ2FpbhIFGgMKAQAKJAoObmF0aXZlLWNvdW50cnkSEgoQCg4gVW5pdGVkLVN0YXRlcwoiCgpvY2N1cGF0aW9uEhQKEgoQIEZhcm1pbmctZmlzaGluZwoWCg1lZHVjYXRpb24tbnVtEgUaAwoBCQopCg5tYXJpdGFsLXN0YXR1cxIXChUKEyBNYXJyaWVkLWNpdi1zcG91c2UKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIFdoaXRlChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA9a9BQoUCgVsYWJlbBILCgkKByA8PTUwSy4KDAoDYWdlEgUaAwoBJgoZCgllZHVjYXRpb24SDAoKCgggSFM

In [12]:
encoded[0]

b'CoEDChQKBWxhYmVsEgsKCQoHIDw9NTBLLgoMCgNhZ2USBRoDCgEZChYKCWVkdWNhdGlvbhIJCgcKBSAxMXRoChcKDmhvdXJzLXBlci13ZWVrEgUaAwoBKAoQCgNzZXgSCQoHCgUgTWFsZQoeCgxyZWxhdGlvbnNoaXASDgoMCgogT3duLWNoaWxkChUKDGNhcGl0YWwtZ2FpbhIFGgMKAQAKJAoObmF0aXZlLWNvdW50cnkSEgoQCg4gVW5pdGVkLVN0YXRlcwokCgpvY2N1cGF0aW9uEhYKFAoSIE1hY2hpbmUtb3AtaW5zcGN0ChYKDWVkdWNhdGlvbi1udW0SBRoDCgEHCiQKDm1hcml0YWwtc3RhdHVzEhIKEAoOIE5ldmVyLW1hcnJpZWQKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIEJsYWNrChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA/LrDQ=='

#### `post_msg` , a file whose JSON formated contents are part of inference request to be sent over REST api.  The binary bytes (ex: serialization of structured data)  have to be Base64 encoded. 

In [13]:
!cat post_msg

{
"signature_name": "predict",
"inputs":[
{"b64":"CoEDChQKBWxhYmVsEgsKCQoHIDw9NTBLLgoMCgNhZ2USBRoDCgEZChYKCWVkdWNhdGlvbhIJCgcKBSAxMXRoChcKDmhvdXJzLXBlci13ZWVrEgUaAwoBKAoQCgNzZXgSCQoHCgUgTWFsZQoeCgxyZWxhdGlvbnNoaXASDgoMCgogT3duLWNoaWxkChUKDGNhcGl0YWwtZ2FpbhIFGgMKAQAKJAoObmF0aXZlLWNvdW50cnkSEgoQCg4gVW5pdGVkLVN0YXRlcwokCgpvY2N1cGF0aW9uEhYKFAoSIE1hY2hpbmUtb3AtaW5zcGN0ChYKDWVkdWNhdGlvbi1udW0SBRoDCgEHCiQKDm1hcml0YWwtc3RhdHVzEhIKEAoOIE5ldmVyLW1hcnJpZWQKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIEJsYWNrChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA/LrDQ=="},
{"b64":"CoUDChwKDHJlbGF0aW9uc2hpcBIMCgoKCCBIdXNiYW5kChUKDGNhcGl0YWwtZ2FpbhIFGgMKAQAKJAoObmF0aXZlLWNvdW50cnkSEgoQCg4gVW5pdGVkLVN0YXRlcwoiCgpvY2N1cGF0aW9uEhQKEgoQIEZhcm1pbmctZmlzaGluZwoWCg1lZHVjYXRpb24tbnVtEgUaAwoBCQopCg5tYXJpdGFsLXN0YXR1cxIXChUKEyBNYXJyaWVkLWNpdi1zcG91c2UKFQoMY2FwaXRhbC1sb3NzEgUaAwoBAAoSCgRyYWNlEgoKCAoGIFdoaXRlChkKCXdvcmtjbGFzcxIMCgoKCCBQcml2YXRlChEKBmZubHdndBIHGgUKA9a9BQoUCgVsYWJlbBILCgkKByA8PTUwSy4KDAoDYW

#### Use `curl` to send inference request to the model serviced by tensorflow model server over REST api

In [14]:
!curl -d @post_msg -X POST http://tfserving:8501/v1/models/census:predict

{
    "outputs": {
        "logistic": [
            [
                0.358853
            ],
            [
                0.554862
            ],
            [
                0.321336
            ]
        ],
        "class_ids": [
            [
                0
            ],
            [
                1
            ],
            [
                0
            ]
        ],
        "probabilities": [
            [
                0.641147,
                0.358853
            ],
            [
                0.445138,
                0.554862
            ],
            [
                0.678664,
                0.321336
            ]
        ],
        "classes": [
            [
                "0"
            ],
            [
                "1"
            ],
            [
                "0"
            ]
        ],
        "logits": [
            [
                -0.580347
            ],
            [
                0.220337
            ],
            [
               

In [15]:
!curl -d @post_msg -X POST http://localhost:8501/v1/models/census:predict

curl: (7) Failed to connect to localhost port 8501: Connection refused


In [23]:
!pwd

/notebooks/tmp/census
